In [ ]:
%pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model.save('./model/all-mpnet-base-v2')


In [ ]:

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential



ml_client = MLClient.from_config(
    credential=DefaultAzureCredential(),
    path="./config.json"
    )


print(ml_client.workspace_name)


In [ ]:
from azure.ai.ml.entities import Model

model = Model(
    path="./model/all-mpnet-base-v2",
    name="all-mpnet-base-v2",
    description="Hugging Face sentence-transformers model"
)
ml_client.models.create_or_update(model)


In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment



env = Environment(
    name="sentence-transformers-env",
    conda_file="./conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04"
)
ml_client.environments.create_or_update(env)


In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, CodeConfiguration

import uuid
endpoint_name = "mpnet-endpoint-2" + str(uuid.uuid4())[:4]

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode="key"
)
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

code_config = CodeConfiguration(
    code="./src/",
    scoring_script="score.py"
)

deployment = ManagedOnlineDeployment(
    name=endpoint.name,
    endpoint_name=endpoint.name,
    model=model,
    environment=env,
    code_configuration=code_config,
    instance_type="Standard_DS3_v2",
    instance_count=1
)
ml_client.online_deployments.begin_create_or_update(deployment).result()
